# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21097003


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:06<01:36,  3.44s/it]

Rendering models:  10%|█         | 3/30 [00:07<01:06,  2.46s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:46,  1.77s/it]

Rendering models:  20%|██        | 6/30 [00:07<00:30,  1.26s/it]

...

Rendering models:  70%|███████   | 21/30 [00:08<00:01,  6.89it/s]

Rendering models:  83%|████████▎ | 25/30 [00:08<00:00,  8.92it/s]

Rendering models:  90%|█████████ | 27/30 [00:08<00:00,  9.48it/s]

Rendering models:  97%|█████████▋| 29/30 [00:09<00:00,  9.20it/s]

aliabdulhameed                        0.057166
ethanth                               0.001213
ellivc                                0.000398
not-logged-in-db23dc032fc427a4e4d9    0.002864
not-logged-in-085368c0fc90e59be41c    0.161238
whitelaughter                         0.000763
Quarkgc                               0.000224
Person200                             0.000379
ElisabethB                            0.236508
sn346808                              0.000854
jheadlee                              0.000516
Arbitrationer                         0.001870
not-logged-in-1a6b4bed55628d6003a3    0.004638
not-logged-in-ffc5c906399855f5ef0f    0.000530
not-logged-in-154b91cc2bc572b9f687    0.001923
not-logged-in-325f47138a9448dfca15    0.043768
Donihee                               0.002688
umtuck1                               0.000743
AAbattery                             0.000487
not-logged-in-3505b6e1760cf8d37fee    0.000785
not-logged-in-df86a718f8f23d575542    0.091755
tungster24   

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())